# CustomerSupport-Agent: AI Customer Service Chatbot

## Overview
This notebook demonstrates the Customer Support Agent featuring:
- LangGraph-based conversation flow
- Long-term and short-term memory
- ChromaDB FAQ knowledge base
- Sentiment analysis with frustration detection
- Real-time WebSocket chat

---

## 1. Installation & Setup

In [ ]:
# Install required packages
!pip install -q langgraph langchain langchain-openai chromadb sentence-transformers textblob nltk

In [ ]:
import os
import sys
from pathlib import Path

# Add project to path
sys.path.insert(0, '../../projects/agents/CustomerSupport-Agent')

# Set up environment
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'  # Replace with actual key

print("✅ Setup complete!")

## 2. Initialize the Support Agent

In [ ]:
from src.conversation.support_agent import get_support_agent
from src.knowledge.faq_store import FAQStore
from src.memory.conversation_memory import ConversationMemory
from src.sentiment.analyzer import SentimentAnalyzer
from src.tools.support_tools import TicketStore

print("🤖 Initializing Customer Support Agent...\n")

# 1. Initialize FAQ knowledge base
faq_store = FAQStore()
print("✅ FAQ knowledge base initialized")

# 2. Initialize conversation memory
memory = ConversationMemory()
print("✅ Conversation memory initialized")

# 3. Initialize sentiment analyzer
sentiment_analyzer = SentimentAnalyzer()
print("✅ Sentiment analyzer initialized")

# 4. Initialize ticket store
ticket_store = TicketStore()
print("✅ Ticket store initialized")

# 5. Create support agent
agent = get_support_agent()
print("✅ Support agent created")

print("\n🎉 All components initialized successfully!")

## 3. Load FAQ Knowledge Base

In [ ]:
# Sample FAQ data
sample_faqs = [
    {
        "question": "How do I reset my password?",
        "answer": "To reset your password, go to Settings > Security > Password and click 'Reset Password'. You'll receive an email with a reset link.",
        "category": "Account"
    },
    {
        "question": "What payment methods do you accept?",
        "answer": "We accept all major credit cards (Visa, MasterCard, American Express), PayPal, and bank transfers.",
        "category": "Billing"
    },
    {
        "question": "How can I contact customer support?",
        "answer": "You can reach our support team 24/7 via live chat, email at support@example.com, or phone at 1-800-SUPPORT.",
        "category": "Contact"
    },
    {
        "question": "What is your refund policy?",
        "answer": "We offer a 30-day money-back guarantee on all plans. If you're not satisfied, contact support for a full refund.",
        "category": "Billing"
    },
    {
        "question": "How do I upgrade my plan?",
        "answer": "Go to Settings > Billing > Plan and select 'Upgrade'. You can upgrade or downgrade at any time.",
        "category": "Account"
    }
]

print(f"📚 Loading {len(sample_faqs)} FAQs into knowledge base...")

# Add FAQs to the store
for faq in sample_faqs:
    faq_store.add_faq(faq['question'], faq['answer'], faq['category'])

print(f"✅ Successfully loaded {len(sample_faqs)} FAQs!")

## 4. FAQ Search

In [ ]:
# Test FAQ search
test_queries = [
    "reset password",
    "payment methods",
    "refund policy",
    "contact support"
]

print("🔍 Testing FAQ Search\n")
print("=" * 60)

for query in test_queries:
    results = faq_store.search(query, top_k=2)
    
    print(f"\n❓ Query: '{query}'")
    print(f"Found {len(results)} results:")
    
    for i, result in enumerate(results, 1):
        print(f"\n  {i}. {result['question']}")
        print(f"     {result['answer'][:80]}...")
        print(f"     Category: {result['category']} | Score: {result['score']:.2f}")

## 5. Sentiment Analysis

In [ ]:
# Test sentiment analysis
test_messages = [
    "Hi, I need help with my account.",
    "This is terrible! I'm very angry!",
    "I'm frustrated and this is useless.",
    "Thanks for your help!",
    "I'm really disappointed with the service."
]

print("😊 Sentiment Analysis Demo\n")
print("=" * 60)

for msg in test_messages:
    sentiment = sentiment_analyzer.analyze(msg)
    
    emoji = "😊" if sentiment['sentiment'] == 'positive' else "😠" if sentiment['sentiment'] == 'negative' else "😐"
    
    print(f"\n{emoji} Message: {msg}")
    print(f"   Sentiment: {sentiment['sentiment']}")
    print(f"   Polarity: {sentiment['polarity']:.2f}")
    print(f"   Frustration: {sentiment['frustration_score']:.2f}")
    
    if sentiment['escalate']:
        print(f"   ⚠️  ESCALATION RECOMMENDED")

## 6. Conversation Flow

In [ ]:
# Simulate a conversation
user_id = "user_123"
session_id = "session_456"

print("💬 Conversation Simulation\n")
print("=" * 60)

# User messages
messages = [
    "Hi, I need help with my account",\n    "I forgot my password and can't log in",\n    "Where do I go to reset it?"
]

for i, msg in enumerate(messages, 1):
    print(f"\n--- Turn {i} ---")
    print(f"User: {msg}")
    
    # Process message through agent
    response = agent.process_message(
        user_id=user_id,
        session_id=session_id,
        message=msg
    )
    
    print(f"\nAgent: {response['message']}")
    print(f"Intent: {response['intent']}")
    print(f"Sentiment: {response['sentiment']}")
    print(f"Escalated: {response['escalated']}")
    
    if response.get('ticket_id'):
        print(f"Ticket Created: {response['ticket_id']}")
    
    if response.get('sources'):
        print(f"Sources: {len(response['sources'])} FAQs used")

## 7. Memory Management

In [ ]:
# Test conversation memory
print("🧠 Conversation Memory Demo\n")

# Store conversation history
conversation = [
    {"role": "user", "content": "I have a billing issue"},
    {"role": "assistant", "content": "I can help with billing. What's the issue?"},
    {"role": "user", "content": "I was charged twice this month"},
    {"role": "assistant", "content": "I apologize for the error. Let me check your account."},
]

session_id = "test_session_memory"

# Add conversation to memory
for msg in conversation:
    memory.add_message(session_id, msg['role'], msg['content'])

print(f"✅ Added {len(conversation)} messages to memory")

# Retrieve conversation history
history = memory.get_conversation_history(session_id)

print(f"\n📜 Conversation History:")
for i, msg in enumerate(history[-5:], 1):  # Show last 5
    print(f"  {i}. [{msg['role'].upper()}] {msg['content'][:50]}...")

# Get conversation summary
summary = memory.get_summary(session_id)
print(f"\n📝 Summary: {summary}")

## 8. Ticket Creation

In [ ]:
# Create support tickets
print("🎫 Ticket Creation Demo\n")

# Sample tickets
ticket_data = [
    {
        "user_id": "user_001",
        "issue": "Unable to access account after password reset",
        "priority": "high"
    },
    {
        "user_id": "user_002",
        "issue": "Question about billing cycle",
        "priority": "low"
    },
    {
        "user_id": "user_003",
        "issue": "Payment failed but money deducted",
        "priority": "urgent"
    }
]

created_tickets = []

for data in ticket_data:
    ticket = ticket_store.create_ticket(
        user_id=data['user_id'],
        issue=data['issue'],
        priority=data['priority']
    )
    created_tickets.append(ticket)
    
    print(f"✅ Ticket Created: {ticket['id']}")
    print(f"   User: {ticket['user_id']}")
    print(f"   Issue: {ticket['issue']}")
    print(f"   Priority: {ticket['priority']}")
    print(f"   Status: {ticket['status']}\n")

In [ ]:
# List all tickets
print(f"📋 All Tickets in System: {ticket_store.get_ticket_count()} total\n")

all_tickets = ticket_store.get_all_tickets()

for ticket in all_tickets:
    status_emoji = "🟢" if ticket['status'] == 'open' else "🟡" if ticket['status'] == 'in_progress' else "🔴"
    print(f"{status_emoji} {ticket['id']} | Priority: {ticket['priority']:8} | {ticket['issue'][:40]}...")

## 9. Full Conversation Flow with Escalation

In [ ]:
# Simulate a conversation that escalates
user_id = "frustrated_user"
session_id = "escalation_session"

print("💬 Frustrated User Conversation\n")
print("=" * 60)

escalation_conversation = [
    "I need help with my account!",
    "I've been waiting for hours!",
    "This is ridiculous! I'm very angry!",
    "Your service is terrible and useless!"
]

for i, msg in enumerate(escalation_conversation, 1):
    print(f"\n--- Turn {i} ---")
    print(f"User: {msg}")
    
    response = agent.process_message(
        user_id=user_id,
        session_id=session_id,
        message=msg
    )
    
    print(f"\nAgent: {response['message']}")
    print(f"Sentiment: {response['sentiment']} (Polarity: {response['polarity']:.2f})")
    print(f"Frustration: {response['frustration_score']:.2f}")
    print(f"Escalated: {response['escalated']}")
    
    if response['escalated']:
        print("\n⚠️  AGENT ESCALATED TO HUMAN SUPPORT")
        if response.get('ticket_id'):
            print(f"🎫 Urgent ticket created: {response['ticket_id']}")

## 10. Interactive Chat

In [ ]:
def interactive_support_chat():
    """Interactive chat with the support agent."""
    print("💬 Customer Support Chat")
    print("Type 'quit' to exit\n")
    
    user_id = input("Enter your user ID: ")
    session_id = f"session_{hash(user_id)}"
    
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("Thank you for using our support! Goodbye! 👋")
            break
        
        if not user_input.strip():
            continue
        
        response = agent.process_message(
            user_id=user_id,
            session_id=session_id,
            message=user_input
        )
        
        print(f"\nAgent: {response['message']}")
        
        if response['escalated']:
            print("⚠️  Connecting you to a human agent...")

# Uncomment to run interactive chat
# interactive_support_chat()

## Summary

### ✅ What We Demonstrated:

1. **Agent Initialization** - Set up LangGraph-based agent
2. **FAQ Knowledge Base** - Vector search for quick answers
3. **Sentiment Analysis** - Detect emotion and frustration
4. **Conversation Flow** - Multi-turn dialogue with context
5. **Memory Management** - Short-term and long-term memory
6. **Ticket Creation** - Automatic escalation system
7. **Escalation Logic** - Sentiment-driven handoff to humans

### 🎯 Key Features:

- ✅ LangGraph conversation state machine
- ✅ ChromaDB FAQ knowledge base
- ✅ Sentiment & frustration analysis
- ✅ Multi-level memory system
- ✅ Automatic ticket creation
- ✅ Smart escalation logic
- ✅ WebSocket real-time support

### 📚 Next Steps:

- Deploy with FastAPI: `uvicorn src.api.main:app --reload`
- Test WebSocket endpoint
- Customize frustration keywords
- Add more FAQs to knowledge base
- Integrate with CRM system

---

**📖 Documentation:** [CustomerSupport-Agent README](../../projects/agents/CustomerSupport-Agent/README.md)